# Unit 7: Content-based Filtering for Rating Prediction using a Factorization Machine

In this section, we switch from collaborative to content-based filtering. Where collaborative filtering exploits similarities among interactions, content-based filtering exploits similarities between user and/or item features. It finds combinations of user-item features that help to predict ratings or rankings.

However, we discussed the superiority of the ranking approach before, for simplicity we do rating prediction again here. The rating predictions are hence used to impose an ordering on items that are then recommended to the user.

The model we use for the relationship between features and ratings is a factorization machine which is similar to matrix factorization and offers more flexibility in modeling.

In [ ]:
from collections import OrderedDict
import itertools
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyfm import pylibfm
from scipy import sparse
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
from recsys_training.data import Dataset, genres, get_user_profiles
from recsys_training.evaluation import get_relevant_items

In [ ]:
ml100k_ratings_filepath = '../data/raw/ml-100k/u.data'
ml100k_item_filepath = '../data/raw/ml-100k/u.item'
ml100k_user_filepath = '../data/raw/ml-100k/u.user'

## Load Data

In [ ]:
data = Dataset(ml100k_ratings_filepath)
data.rating_split(seed=42)
user_ratings = data.get_user_ratings()

In [ ]:
item_feat = pd.read_csv(ml100k_item_filepath, sep='|', header=None,
                        names=['item', 'title', 'release', 'video_release', 'imdb_url']+genres,
                        engine='python')

In [ ]:
user_feat = pd.read_csv(ml100k_user_filepath, sep='|', header=None,
                        names=['user', 'age', 'gender', 'occupation', 'zip'])

## User and Item Content (Features)

### Exploration

In [ ]:
item_feat.head()

In [ ]:
user_feat.head()

### Preprocessing

#### Items

We keep the following information for items:
* release year
* genres

In [ ]:
def min_max_scale(val, bounds):
    min_max_range = bounds['max']-bounds['min']
    return (val-bounds['min'])/min_max_range

In [ ]:
# Infer the release year
idxs = item_feat[item_feat['release'].notnull()].index
item_feat.loc[idxs, 'release_year'] = item_feat.loc[idxs, 'release'].str.split('-')
item_feat.loc[idxs, 'release_year'] = item_feat.loc[idxs, 'release_year'].apply(lambda val: val[2]).astype(int)

# Impute median release year value for the items with missing release year
top_year = item_feat.loc[idxs, 'release_year'].astype(int).describe()['50%']
idx = item_feat[item_feat['release'].isnull()].index
item_feat.loc[idx, 'release_year'] = top_year

# Min-max scale the release year
item_year_bounds = {'min': item_feat['release_year'].min(),
                    'max': item_feat['release_year'].max()}
item_feat['release_year'] = item_feat['release_year'].apply(
    lambda year: min_max_scale(year, item_year_bounds))

# Drop other columns
item_feat.drop(['title', 'release', 'video_release', 'imdb_url'], axis=1, inplace=True)

In [ ]:
item_feat.head()

#### users

We keep the following information for users:
* age
* gender
* occupation
* zip-code

In [ ]:
# Min-max scale the age
user_age_bounds = {'min': user_feat['age'].min(),
                   'max': user_feat['age'].max()}
user_feat['age'] = user_feat['age'].apply(lambda age: min_max_scale(age, user_age_bounds))

# Transform gender characters to numerical values (categories)
genders = sorted(user_feat['gender'].unique())
user_gender_map = dict(zip(genders, range(len(genders))))
user_feat['gender'] = user_feat['gender'].map(user_gender_map)

# Transform occupation strings to numerical values (categories)
occupations = sorted(user_feat['occupation'].unique())
user_occupation_map = dict(zip(occupations, range(len(occupations))))
user_feat['occupation'] = user_feat['occupation'].map(user_occupation_map)

# Transform the zip codes to categories keeping the first three digits and impute for missing
idxs = user_feat[~user_feat['zip'].str.isnumeric()].index
user_feat.loc[idxs, 'zip'] = '00000'
zip_digits_to_cut = 3
user_feat['zip'] = user_feat['zip'].apply(lambda val: int(val) // 10 ** zip_digits_to_cut)

![](Parrot.png)

**TODO:** Complete `get_user_profiles` to infer user profiles combining their ratings with the item features the users liked

In addition, we infer profiles by combining item information with rating data for each user to get features that represent the users' preferred genres and film age

In [ ]:
def user_profiler(group):
    genre_dist = group[genres].mean()
    year_dist = group['release_year'].describe()[['mean', 'std', '50%']]

    return pd.concat((genre_dist, year_dist), axis=0)

In [ ]:
def get_user_profiles(ratings: pd.DataFrame,
                      item_feat: pd.DataFrame,
                      min_rating: float = 4.0) -> pd.DataFrame:
    pass
    
    return profiles

Finally, we join the original user information with their profiles' information and one-hot-encode categorical information

In [ ]:
profiles = get_user_profiles(data.train_ratings, item_feat)
user_feat = user_feat.merge(profiles, on='user', how='left')

occupation_1H = pd.get_dummies(user_feat['occupation'], prefix='occupation')
zip_1H = pd.get_dummies(user_feat['zip'], prefix='zip')

user_feat.drop(['occupation', 'zip', ], axis=1, inplace=True)
user_feat = pd.concat([user_feat, occupation_1H, zip_1H], axis=1)

user_feat.fillna(0, inplace=True)

We remove the user/item id columns and replace the current dataframe indices with their values

In [ ]:
user_feat.index = user_feat['user'].values
user_feat.drop('user', axis=1, inplace=True)

item_feat.index = item_feat['item'].values
item_feat.drop('item', axis=1, inplace=True)

### Final Check

In [ ]:
item_feat.head()

In [ ]:
user_feat.head()

Sparsity of user/item content information

In [ ]:
(user_feat==0).sum().sum()/user_feat.size

In [ ]:
(item_feat==0).sum().sum()/item_feat.size

## Factorization Machine for a Content-based Recommender

![FM](fm.png)

[Steffen Rendle: Factorization Machines](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)

[pyFM - Factorization Machines in Python](https://github.com/coreylynch/pyFM)

#### Create Feature Matrices

In [ ]:
# fetch content information for all observed user-item rating combinations
user_cb_feat_train = user_feat.loc[data.train_ratings.user.values].values
user_cb_feat_test = user_feat.loc[data.test_ratings.user.values].values
item_cb_feat_train = item_feat.loc[data.train_ratings.item.values].values
item_cb_feat_test = item_feat.loc[data.test_ratings.item.values].values

In [ ]:
# concatenate user and item content information to form design matrices
# and convert to sparse matrix in Compressed Sparse Row (CSR) format
X_train = np.concatenate((user_cb_feat_train, item_cb_feat_train), axis=1)
X_train = sparse.csr_matrix(X_train)
X_test = np.concatenate((user_cb_feat_test, item_cb_feat_test), axis=1)
X_test = sparse.csr_matrix(X_test)

In [ ]:
def get_sparsity(sparse_arr) -> float:
    num_elements = sparse_arr.shape[0]*sparse_arr.shape[1]
    num_nonzero_elements = sparse_arr.nnz
    density = num_nonzero_elements/num_elements
    return 1-density

In [ ]:
X_train

In [ ]:
# Sparsity of Training Data
get_sparsity(X_train)

In [ ]:
X_test

In [ ]:
# Sparsity of Test Data
get_sparsity(X_test)

#### Create Target Matrices for Rating Predictions

In [ ]:
y_train = data.train_ratings.rating.values.astype(float)
y_test = data.test_ratings.rating.values

#### Train Factorization Machine for Rating Prediction as Regressor using pyFM

In [ ]:
n_epochs = 10  # number of full stochastic passes through the training data
k = 16
random_seed = 28

In [ ]:
fm_cb = pylibfm.FM(num_factors=k,
                   num_iter=n_epochs,
                   verbose=True,
                   task="regression",
                   initial_learning_rate=0.001,
                   learning_rate_schedule="optimal",
                   seed=random_seed)
fm_cb.fit(X_train, y_train)

### Evaluation on Test Set

In [ ]:
y_pred = fm_cb.predict(X_test)

$MSE$

In [ ]:
mean_squared_error(y_test, y_pred)

$MAE$

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
def get_prediction(fm: object, user: int, user_feat: pd.DataFrame, item_feat: pd.DataFrame,
                   items: np.array = None, remove_known_pos: bool = True) -> Dict[int, Dict[str, float]]:
    
    if items is None:
        if remove_known_pos:
            # Predict from unobserved items
            known_items = np.array(list(user_ratings[user].keys()))
            items = np.setdiff1d(data.items, known_items)
        else:
            items = np.array(data.items)
    if type(items) == np.int64:
        items = np.array([items])
    
    n_items = len(items)
    
    single_user_cb_feat = user_feat.loc[user].values.reshape(1, -1).repeat(n_items, axis=0)
    all_items_cb_feat = item_feat.loc[items].values
    
    input_data = np.concatenate((single_user_cb_feat, all_items_cb_feat), axis=1)
    input_data = sparse.csr_matrix(input_data)
    
    preds = fm.predict(input_data)
    sorting = np.argsort(preds)[::-1]
    
    preds = {item: {'pred': pred} for item, pred in
             zip(items[sorting], preds[sorting])}
    
    return preds

In [ ]:
predictions = get_prediction(fm_cb, 1, user_feat, item_feat)
list(predictions.items())[:20]

In [ ]:
def get_recommendations(fm_cb: object, user: int, N: int, user_feat: pd.DataFrame, item_feat: pd.DataFrame,
                        remove_known_pos: bool = True) -> List[Tuple[int, Dict[str, float]]]:
    predictions = get_prediction(fm_cb, user, user_feat, item_feat,
                                 remove_known_pos=remove_known_pos)
    recommendations = []
    # TODO: Simplify
    for item, pred in predictions.items():
        add_item = (item, pred)
        recommendations.append(add_item)
        if len(recommendations) == N:
            break

    return recommendations

In [ ]:
get_recommendations(fm_cb, 1, N=10, user_feat=user_feat, item_feat=item_feat)

## Evaluation

In [ ]:
N = 10

In [ ]:
relevant_items = get_relevant_items(data.test_ratings)

In [ ]:
users = relevant_items.keys()
prec_at_N = dict.fromkeys(data.users)

for user in users:
    recommendations = get_recommendations(fm_cb, user, N, user_feat=user_feat, item_feat=item_feat, remove_known_pos=True)
    recommendations = [val[0] for val in recommendations]
    hits = np.intersect1d(recommendations,
                          relevant_items[user])
    prec_at_N[user] = len(hits)/N

In [ ]:
recommendations

In [ ]:
np.mean([val for val in prec_at_N.values() if val is not None])